In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
import json
from playwright.async_api import async_playwright
import time

In [3]:
#start the browser
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [4]:
#scrap the page for 2023 questions only
url = "https://sansad.in/ls/questions/questions-and-answers"
await page.goto(url)
time.sleep(2)

await page.get_by_role("button", name="Facet Search on Questions").click()
await page.locator("#fromDate").click()
await page.get_by_role("option", name="2014").click()
await page.locator("#toDate").click()
await page.get_by_role("option", name="2023").click()
await page.get_by_role("button", name="button to search by from and to dates").click()

await page.get_by_role("button", name="10").click()
await page.get_by_role("option", name="100").click()

await page.locator('#rows-per-page').click()
await page.locator('#menu- > div.MuiPaper-root.MuiPaper-elevation.MuiPaper-rounded.MuiPaper-elevation1.MuiPaper-root.MuiMenu-paper.MuiPaper-elevation.MuiPaper-rounded.MuiPaper-elevation8.MuiPopover-paper.mui-style-d9lhs1 > ul > li:nth-child(6)').click()

time.sleep(2)

#creating an empty variable called p_list
p_list=[]

#looing through the website data page-wis and then parsing it in beautiful soup and adding it to p_list variable
for i in range(0,5):
    html = await page.content()
    soup = BeautifulSoup(html)
    time.sleep(2)

    elements = soup.findAll(class_='MuiTableRow-root mui-style-pfw9qk')

    p_list.append(elements)

    await page.get_by_role("button", name="Go to next page").click()


In [5]:
p_list

[[<tr class="MuiTableRow-root mui-style-pfw9qk"><td class="MuiTableCell-root MuiTableCell-body MuiTableCell-sizeSmall mui-style-1l45qeu"><div><div><span><span class="">Allocation of Foodgrains to Rajasthan</span></span></div><div style="background-color: rgb(229, 245, 255); display: inline-flex; color: rgb(27, 110, 194); text-transform: uppercase; font-size: 0.675rem; font-weight: 500; padding: 0.5%; border-radius: 0.15rem; margin-top: 0.5%; cursor: default;"></div></div></td><td class="MuiTableCell-root MuiTableCell-body MuiTableCell-sizeSmall mui-style-1l45qeu"><div><span><span class="">17</span></span></div></td><td class="MuiTableCell-root MuiTableCell-body MuiTableCell-sizeSmall mui-style-1l45qeu"><div><span><span class="">XI</span></span></div></td><td class="MuiTableCell-root MuiTableCell-body MuiTableCell-sizeSmall mui-style-1l45qeu"><div><span><span class="">2023-04-05</span></span></div></td><td class="MuiTableCell-root MuiTableCell-body MuiTableCell-sizeSmall mui-style-1l45q

In [6]:
#looped thrice: 1 and 2 to get into the lists and the 3rd one to differnciate the individual cell entries
#saved the new list in table_list[], which has two lists: 1 is for the entire row and the second is for each entry/cell in the row


table_list = []
for table in p_list:
   # print(row)
    # print("====")
    for row in table:
        row_list = []
        for cell in row:
            row_list.append(cell.text)
        table_list.append(row_list)
print(table_list)    


[['Allocation of Foodgrains to Rajasthan', '17', 'XI', '2023-04-05', 'Unstarred', '5441', 'CONSUMER AFFAIRS, FOOD AND PUBLIC DISTRIBUTION', 'Bhagirath Choudhary', '', ''], ['Installation of Nuclear Reactors', '17', 'XI', '2023-04-05', 'Starred', '480', 'ATOMIC ENERGY', 'Rajendra Dhedya Gavit', '', ''], ['Provision of New Trains', '17', 'XI', '2023-04-05', 'Starred', '471', 'RAILWAYS', 'Pratap Chandra Sarangi', '', ''], ['National Digital Literacy Mission', '17', 'XI', '2023-04-05', 'Unstarred', '5436', 'ELECTRONICS AND INFORMATION TECHNOLOGY', 'Kalaben Mohanbhai DelkarGajanan Chandrakant KirtikarHemant Tukaram Godse', '', ''], ['Modern Regenerative Braking System Technology', '17', 'XI', '2023-04-05', 'Starred', '470', 'RAILWAYS', 'Dhairyasheel S. ManeShrirang Appa Barne', '', ''], ['Status of Progress of 5G Services', '17', 'XI', '2023-04-05', 'Unstarred', '5431', 'COMMUNICATIONS', 'Venkata Satyavathi Beesetti', '', ''], ['Setting up of New Industries in Rajasthan', '17', 'XI', '2023-

In [8]:
# convert table_list into dataframe

df = pd.DataFrame(table_list, columns = ['Title_of_debate', 'Lok_Sabha_Number', 'Session_number', 'Date', 'Question_type', 'Question_number', 'Ministry', 'Members', 'Empty_1', 'Empty_2'])

In [9]:
df

,Title_of_debate,Lok_Sabha_Number,Session_number,Date,Question_type,Question_number,Ministry,Members,Empty_1,Empty_2
0,Allocation of Foodgrains to Rajasthan,17,XI,2023-04-05,Unstarred,5441,"CONSUMER AFFAIRS, FOOD AND PUBLIC DISTRIBUTION",Bhagirath Choudhary,,
1,Installation of Nuclear Reactors,17,XI,2023-04-05,Starred,480,ATOMIC ENERGY,Rajendra Dhedya Gavit,,
2,Provision of New Trains,17,XI,2023-04-05,Starred,471,RAILWAYS,Pratap Chandra Sarangi,,
3,National Digital Literacy Mission,17,XI,2023-04-05,Unstarred,5436,ELECTRONICS AND INFORMATION TECHNOLOGY,Kalaben Mohanbhai DelkarGajanan Chandrakant Ki...,,
4,Modern Regenerative Braking System Technology,17,XI,2023-04-05,Starred,470,RAILWAYS,Dhairyasheel S. ManeShrirang Appa Barne,,
...,...,...,...,...,...,...,...,...,...,...
495,Generation of Livelihood for Youths and Women ...,17,XI,2023-04-03,Unstarred,5269,DEVELOPMENT OF NORTH EASTERN REGION,Pallab Lochan Das,,
496,Status of Block in Odisha,17,XI,2023-04-03,Unstarred,5266,TRIBAL AFFAIRS,Suresh Kumar Pujari,,
497,Central University in Dharamshala,17,XI,2023-04-03,Unstarred,5265,Education,Kishan Kapoor,,
498,Off-Budget Borrowing,17,XI,2023-04-03,Unstarred,5263,FINANCE,Srinivas Reddy ManneKomati Reddy Venkat ReddyY...,,
